In [41]:
# import necessary modules
import numpy as np
import pandas as pd
from random import random, seed
from math import exp

In [42]:
# Initialize Network
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create network and layers as lists
    network = []
    # Create the hidden and output layer where neurons are saved as dictionaries
    hidden_layer = [{"weights":[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    output_layer = [{"weights":[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

In [43]:
# Calculate neuron activation for an input
def activate_neuron(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [44]:
# transfer neuron
def transfer_neuron_sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation))

In [45]:
# forward propagate inputs to a network output
def forward_propagate(network, row):
    inputs = row
    for i, layer in enumerate(network):
        net_inputs = []
        for j, neuron in enumerate(layer):
            activation = activate_neuron(neuron["weights"], inputs)
            neuron["output"] = transfer_neuron_sigmoid(activation)
            net_inputs.append(neuron["output"])
        inputs = net_inputs
    return inputs        

In [46]:
# Calculate the derivative of an neuron output
def transfer_derivative_sigmoid(output):
    return output * (1.0 - output)

In [47]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in range(len(network)-1, -1, -1):
        layer = network[i]
        errors, error_signals = [],[]
        
        for j, neuron in enumerate(layer):
            if i == len(network) -1:
                error = neuron["output"] - expected[j]
                error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                
            else:
                error = 0
                for k, next_neuron in enumerate(network[i+1]):
                    error += next_neuron["weights"][j] * next_neuron["delta"]
                    error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                    errors.append(error)
                    error_signals.append(error_signal) 
            
            errors.append(error)
            error_signals.append(error_signal) 
            neuron['delta'] = error_signal
        

In [49]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        if i == 0:
            inputs = row[:-1]
        else:
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron["weights"][j] -= l_rate * neuron['delta'] * inputs[j]
                neuron["weights"][-1] -= l_rate * neuron['delta']

In [75]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1   
            sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(n_outputs)]) 
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f"epoch:{epoch:2d},  lrate:{l_rate},  error:{sum_error:.4f}")    

In [70]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv'
dataset = pd.read_csv(url)

wheat_seeds = []
for idx, row in dataset.iterrows():
    row_list = []
    for num in row:
        row_list.append(num)
    wheat_seeds.append(row_list)
    
print(np.shape(wheat_seeds))

wheat_inputs = wheat_seeds[:2]
print(wheat_inputs)

(209, 8)
[[14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 1.0], [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 1.0]]


In [71]:
seed(1)

network = initialize_network(7,2,3)
print(network)

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095, 0.4494910647887381, 0.651592972722763, 0.7887233511355132]}, {'weights': [0.0938595867742349, 0.02834747652200631, 0.8357651039198697, 0.43276706790505337, 0.762280082457942, 0.0021060533511106927, 0.4453871940548014, 0.7215400323407826]}], [{'weights': [0.22876222127045265, 0.9452706955539223, 0.9014274576114836]}, {'weights': [0.030589983033553536, 0.0254458609934608, 0.5414124727934966]}, {'weights': [0.9391491627785106, 0.38120423768821243, 0.21659939713061338]}]]


In [77]:
train_network(network, wheat_inputs, 0.5, 20, 3)

epoch: 0,  lrate:0.5,  error:0.0344
epoch: 1,  lrate:0.5,  error:0.0331
epoch: 2,  lrate:0.5,  error:0.0319
epoch: 3,  lrate:0.5,  error:0.0307
epoch: 4,  lrate:0.5,  error:0.0297
epoch: 5,  lrate:0.5,  error:0.0287
epoch: 6,  lrate:0.5,  error:0.0278
epoch: 7,  lrate:0.5,  error:0.0269
epoch: 8,  lrate:0.5,  error:0.0261
epoch: 9,  lrate:0.5,  error:0.0253
epoch:10,  lrate:0.5,  error:0.0245
epoch:11,  lrate:0.5,  error:0.0239
epoch:12,  lrate:0.5,  error:0.0232
epoch:13,  lrate:0.5,  error:0.0226
epoch:14,  lrate:0.5,  error:0.0220
epoch:15,  lrate:0.5,  error:0.0214
epoch:16,  lrate:0.5,  error:0.0209
epoch:17,  lrate:0.5,  error:0.0204
epoch:18,  lrate:0.5,  error:0.0199
epoch:19,  lrate:0.5,  error:0.0194
